In [1]:
import os
from PIL import Image
from tqdm import tqdm
import pandas as pd 

data_root = '/data/wsi/TCTGC50k/TCTGC50k-volume1/'

wsi_names = ['T202234673', 'T202234666', 'T202329508', 'test0001', 'T202232995', 'M202331996']
error_df = 'tct_img_error2.xlsx'
volume_num = 1
if os.path.exists(error_df):
    df = pd.read_excel(error_df)
else:
    df = pd.DataFrame(columns=['wsi_name', 'img_name', 'volume'])
for wsi_name in wsi_names:
    img_files = os.listdir(os.path.join(data_root, wsi_name))
    for img_file in tqdm(img_files):
        try:
            img_path = os.path.join(data_root, wsi_name, img_file)
            img = Image.open(img_path)
            img.resize((224, 224))
        except:
            new_row = pd.DataFrame({'wsi_name': [wsi_name], 'img_name': [img_file], 'volume': [volume_num]})
            df = pd.concat([df, new_row], ignore_index=True)
df.to_excel(error_df, index=False)

100%|██████████| 442/442 [01:33<00:00,  4.73it/s]
